In [3]:
import pandas as pd
E = pd.read_csv('E_norm.csv',encoding='UTF-8')

In [16]:
E.rename(columns={'% r': 'r'}, inplace=True)

In [17]:
columns = E.columns[2:]

In [27]:
import numpy as np
s = columns.size
l = E.iloc[:, :2]
i= 1
E_n = pd.concat([l, E[columns[i]]], axis=1)
column_name = 'es.normE (V/m) @ loc=' + str(i)
E_n.rename(columns={column_name: 'normE'}, inplace=True)
print(E_n['r'].max(axis=0))
r = np.linspace(0, E_n['r'].max(axis=0), 1000)
z = np.linspace(E_n['z'].min(axis=0), E_n['z'].max(axis=0), 1000)
[R,Z] = np.meshgrid(r,z)


0.49995


In [21]:
import matplotlib.pyplot as plt
plt.scatter(E_n['r'], E_n['z'], c=E_n['normE'], cmap='viridis')
plt.colorbar(label='normE')
plt.xlabel('r')
plt.ylabel('z')
plt.title('Color Map of normE')
plt.show()

KeyboardInterrupt: 

In [ ]:
for i in range(0,s-1):
    E_n = pd.concat([l, E[columns[i]]], axis=1)